In [1]:
from zipfile import ZipFile
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import cupy 
# import cudf

In [3]:
df_dev = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'development.csv')
df_eval = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'evaluation.csv')

In [4]:
df_eval.shape

(74999, 5)

In [5]:
df = df_dev.append(df_eval)

In [6]:
# df['date'] = df['date'].\
#     astype('string')\
#     .str.split(' ')\
#     .apply(lambda x : ' '.join([x[i] for i in [1,2,3,5]]))\
#     .pipe(pd.to_datetime)

### Stopwords analysis: sklearn, nltk, stop_words

#### nltk

In [7]:
from nltk import download as nltk_download

nltk_download('stopwords')

from nltk.corpus import stopwords

nltk_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/edoch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### sklearn

In [8]:
from sklearn.feature_extraction import text

sklearn_stopwords = list(text.ENGLISH_STOP_WORDS)

#### stop_words

In [9]:
# from stop_words import get_stop_words

# stop_words_stopwords = get_stop_words('english')

I should try to use both all stopwords, and sklearn and nltk stopwords singularly (and also no stopwords)

In [10]:
def stopwords_list_gen(source_list = [sklearn_stopwords,nltk_stopwords], generate_neg = True):
    stopwords_all_list = set()
    for source in source_list:
        stopwords_all_list = stopwords_all_list.union(set(source))
        if generate_neg:
            stopwords_all_list = stopwords_all_list.union(set([f'{i+"_neg"}' for i in source]))
    return stopwords_all_list

## Preprocessing

Tokenization

In [11]:
df['text_token'] = df['text'].str.split()

Removal of '@words', '&words', 'http:words'

In [12]:
df['text_noAt'] = df['text_token'].apply(lambda x : [i for i in x if not(i.startswith('@'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('&'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('http:'))])

Removal of '&amp', '&quot'

In [13]:
df['text_noAmpQuot'] = df['text_noAt']\
    .apply(lambda x : [i for i in x if '&amp' not in i])\
    .apply(lambda x : [i for i in x if '&quot' not in i])

Removal of repeated letters (incomplete)

In [14]:
a = 'ulaaa'

def remove_repeated_letters(word):
    for letter in word:
        print(letter)

# nltk_stemmer.stem(a)

Remove puntuation

In [15]:
import string as py_string
df['text_noPunct'] = df['text_noAmpQuot']\
    .apply(lambda x : [i.translate(str.maketrans('', '', py_string.punctuation)) for i in x])\
    .apply(lambda x : [i for i in x if i != ''])

Negation

In [16]:
from nltk.sentiment.util import mark_negation

df['text_neg'] = df['text_noPunct'].apply(lambda x : mark_negation(x))

Removal of stopwords

In [17]:
# considered_stopwords = stop_words_stopwords + [f'{word}_neg' for word in stop_words_stopwords]

# df['text_noStopwords'] = df['text_neg'].apply(lambda x : [i for i in x if i not in considered_stopwords])

Stemming and lemmatization

In [18]:
# from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer

# nltk_download('wordnet')
# nltk_download('omw-1.4')

# nltk_stemmer = PorterStemmer()
# # nltk_stemmer = LancasterStemmer()
# # nltk_stemmer = SnowballStemmer('english')
# nltk_lemmatizer = WordNetLemmatizer()
# # from nltk.corpus import wordnet

# df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_stemmer.stem(word) for word in x])
# df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_lemmatizer.lemmatize(word) for word in x])

User manual filter

In [19]:
df = df.loc[
    (~(df['user'] == 'lost_dog') &
    ~(df['user'] == 'webwoke') &
    ~(df['user'] == 'tweetpet') &
    ~(df['user'].str.contains('tweeteradder')) &
    ~(df['user'].str.contains('tweetfollow')) &
    ~(df['user'] == 'divxdownloads')) |
    df['sentiment'].isna()
]

In [20]:
df[df['sentiment'].isna()].shape

(74999, 11)

## Model creation

In [21]:
from sklearn.model_selection import ParameterGrid

linearSVC_params = {
    'penalty' : ['l1'], #['l1','l2']
    'dual' : [False],
    'tol' : [1e-3,1e-4],
    'fit_intercept' : [False],
    'class_weight' : ['balanced'],
    'max_iter' : [100,300],
    'random_state' : [42],
    'C' : [1,10,50]
}

TfidfVectorizer_params = {
    'stop_words' : [None],
    'ngram_range' : [(1,1)],
    'max_features' : [None, 15000],
    'max_df' : [1.0,0.1,0.005],
    'min_df' : [1,0.00001],
    'binary' : [True,False],
    'norm' : ['l1','l2'],
    'use_idf' : [True],
    'smooth_idf' : [True,False],
    'sublinear_tf' : [False]
}

In [22]:
print(len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params))))

print(len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params)))*30/60/60)

1152
9.6


# Train test split

In [23]:
# from sklearn.model_selection import ParameterGrid
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.metrics import f1_score, classification_report, confusion_matrix

# from datetime import datetime
# from pathlib import Path

# logs_path = Path.cwd()/'logs'
# results_path = Path.cwd()/'results'
# logs_path.mkdir(exist_ok=True)
# results_path.mkdir(exist_ok=True)
# log_file_name = logs_path/f'log_out_{datetime.now()}.txt'
# results_file_name = results_path/f'results_out_{datetime.now()}.txt'
# with open(log_file_name, 'w') as file_log:
#     file_log.write('File output\n\n')
# with open(results_file_name, 'w') as file_result:
#     file_result.write('configuration_number,vectorizer_param,model_param,f1\n')

# df_final = df
# # df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

# mask_train_test = df_final['sentiment'].notna()

# X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
# y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
# X_test = df_final.loc[~mask_train_test,:]['text_final'].values

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_train_valid, 
#     y_train_valid, 
#     shuffle=True, 
#     # stratify=y_train_valid, 
#     train_size=0.8, 
#     random_state=50
# )

# configuration_number = 0
# print(f'Number of combinations:\t{len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params)))}')
# for vectorizer_param in ParameterGrid(TfidfVectorizer_params):
#     for model_param in ParameterGrid(linearSVC_params):
#         configuration_number += 1
#         print(configuration_number, vectorizer_param, model_param)

#         pipe = Pipeline([
#             ('tfidf', TfidfVectorizer(**vectorizer_param)),
#             ('linearSVC', LinearSVC(**model_param))
#         ])

#         pipe.fit(X_train, y_train)

#         f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
#         report = classification_report(y_valid, pipe.predict(X_valid))
#         # confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

#         with open(log_file_name, 'a') as file_log:
#             file_log.write(f'configuration_number: {configuration_number}\n')
#             file_log.write(f'vectorizer_param: \t{vectorizer_param}\n')
#             file_log.write(f'model_param: \t\t{model_param}\n')
#             file_log.write(f'f1_score:\t\t\t{f1}\n\n')
#             file_log.write(f'{report}\n\n')
#             file_log.write(f'{"*"*150}\n\n')
#         with open(results_file_name, 'a') as file_result:
#             file_result.write(f'{configuration_number},{vectorizer_param},{model_param},{f1}\n')

# Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

mask_train_test = df_final['sentiment'].notna()

X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
X_test = df_final.loc[~mask_train_test,:]['text_final'].values

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    shuffle=True, 
    # stratify=y_train_valid, 
    train_size=0.9, 
    random_state=42
)

vectorizer = 'tfidf'
model = 'linearSVC'

pipe = Pipeline([
    (vectorizer, TfidfVectorizer()),
    (model, LinearSVC())
])

def params_for_GridSearchCV(params_IN,step_name):
    return {f'{step_name}__{key}':value for (key,value) in params_IN.items()}

params_all = params_for_GridSearchCV(TfidfVectorizer_params,vectorizer)
params_all.update(params_for_GridSearchCV(linearSVC_params,model))
print(params_all)

gscv = GridSearchCV(pipe, params_all, cv = 3, verbose = 4, scoring='f1_macro')

gscv.fit(X_train, y_train)

f1 = f1_score(y_valid, gscv.predict(X_valid),average='macro')
report = classification_report(y_valid, gscv.predict(X_valid))
confusion = confusion_matrix(y_valid, gscv.predict(X_valid))

print(f1)
print(report)
print(confusion)

from pathlib import Path
from datetime import datetime

results_name = Path.cwd()/'gscv_results'/(str(datetime.now()).replace('.','-').replace(':','-')+'.csv')

pd.DataFrame(gscv.cv_results_).to_csv(results_name)

In [33]:
a = pd.read_csv('gscv_results\\2022-01-12 20-43-02-990406.csv')
print(a.loc[a['rank_test_score']==1]['params'].iloc[0])
print(a.loc[a['rank_test_score']==1]['params'].iloc[1])

{'linearSVC__C': 1, 'linearSVC__class_weight': 'balanced', 'linearSVC__dual': False, 'linearSVC__fit_intercept': False, 'linearSVC__max_iter': 100, 'linearSVC__penalty': 'l1', 'linearSVC__random_state': 42, 'linearSVC__tol': 0.001, 'tfidf__binary': True, 'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l1', 'tfidf__smooth_idf': True, 'tfidf__stop_words': None, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True}
{'linearSVC__C': 1, 'linearSVC__class_weight': 'balanced', 'linearSVC__dual': False, 'linearSVC__fit_intercept': False, 'linearSVC__max_iter': 300, 'linearSVC__penalty': 'l1', 'linearSVC__random_state': 42, 'linearSVC__tol': 0.001, 'tfidf__binary': True, 'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l1', 'tfidf__smooth_idf': True, 'tfidf__stop_words': None, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True}


# Best models test

1

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

mask_train_test = df_final['sentiment'].notna()

X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
X_test = df_final.loc[~mask_train_test,:]['text_final'].values

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    shuffle=True, 
    # stratify=y_train_valid, 
    train_size=0.9, 
    random_state=42
)

vectorizer = TfidfVectorizer(
    binary = True, 
    max_df = 1.0, 
    min_df = 1, 
    max_features = None, 
    ngram_range = (1,1), 
    norm = 'l1', 
    smooth_idf = True, 
    stop_words = None, 
    sublinear_tf = False, 
    use_idf = True
)

model = LinearSVC(
    C = 1, 
    class_weight = 'balanced', 
    dual = False, 
    fit_intercept = False, 
    max_iter = 100, 
    penalty = 'l1', 
    random_state = 42, 
    tol = 0.001
)

pipe = Pipeline([
    ('tfidf', vectorizer),
    ('linearSVC', model)
])

pipe.fit(X_train, y_train)

f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
report = classification_report(y_valid, pipe.predict(X_valid))
confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

print(f1)
print(report)
print(confusion)

0.7801731691263916
              precision    recall  f1-score   support

         0.0       0.72      0.79      0.75      9395
         1.0       0.84      0.78      0.81     12900

    accuracy                           0.78     22295
   macro avg       0.78      0.78      0.78     22295
weighted avg       0.79      0.78      0.78     22295

[[ 7436  1959]
 [ 2876 10024]]


# MODIFIED: l2 and 

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

mask_train_test = df_final['sentiment'].notna()

X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
X_test = df_final.loc[~mask_train_test,:]['text_final'].values

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    shuffle=True, 
    # stratify=y_train_valid, 
    train_size=0.9, 
    random_state=42
)

vectorizer = TfidfVectorizer(
    binary = True, 
    max_df = 1.0, 
    max_features = None, 
    min_df = 1, 
    ngram_range = (1,1), 
    norm = 'l1', 
    smooth_idf = True, 
    stop_words = None, 
    sublinear_tf = False, 
    use_idf = True
)

model = LinearSVC(
    C = 1, 
    class_weight = 'balanced', 
    dual = False, 
    fit_intercept = False, 
    max_iter = 100, 
    penalty = 'l2', 
    random_state = 42, 
    tol = 0.001
)

pipe = Pipeline([
    ('tfidf', vectorizer),
    ('linearSVC', model)
])

pipe.fit(X_train, y_train)

f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
report = classification_report(y_valid, pipe.predict(X_valid))
confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

print(f1)
print(report)
print(confusion)

0.7832058576260634
              precision    recall  f1-score   support

         0.0       0.72      0.80      0.76      9395
         1.0       0.84      0.78      0.81     12900

    accuracy                           0.79     22295
   macro avg       0.78      0.79      0.78     22295
weighted avg       0.79      0.79      0.79     22295

[[ 7476  1919]
 [ 2850 10050]]


Results generation

In [45]:
y_pred = pipe.predict(X_test)
pd.DataFrame(y_pred.astype(int),columns=['Predicted']).to_csv("output3_78056.csv",index_label="Id", header=["Predicted"])